# Titanic Dataset Analysis



This analysis is conducted as part of the Udacity data analysis nanodegree. Given the choice between analyzing baseball statistics and Titanic passenger statistics, I chose the Titanic statistics. Given the nature of the data, the obvious question to try and answer is:

__What factors influenced whether or not a given passenger survived?__

In [ ]:
#Some imports
import pandas
import matplotlib

## Data Cleaning